In [ ]:
# --- Single-image predict that REUSES app.predict helpers and shows 6-pack if reps exist ---

from pathlib import Path
import numpy as np
import cv2
import matplotlib.pyplot as plt

from app.config import SimpleConfig
from app.preprocess import Preprocessor
from app.arcface import ArcFaceEmbedder
from app.faiss_vec import FaissVector

# Reuse internal helpers from predict.py (we already rely on these in predict_folder)
from app.predict import (
    _align,
    _apply_pipeline,
    _show_triplet,
    _show_sixpack,
    _load_gallery_reps_from_disk,
)

def predict_one(image_path: str | Path, k: int | None = None, show: bool = True):
    """
    Predict a single image against the FAISS gallery using the same pipeline as predict.py.
    Visualization:
      - If cfg.SAVE_REPS=True and reps exist -> 6-panel (query + gallery reps)
      - else -> 3-panel (query only)
    Returns: (hits, topk_labels)
    """
    p = Path(image_path)
    if not p.exists():
        raise FileNotFoundError(f"Not found: {p}")

    # 1) Load config & components (consistent with your main.py/predict.py)
    cfg = SimpleConfig()
    pre = Preprocessor(cfg)
    emb = ArcFaceEmbedder(cfg.ONNX_PATH)   # if your embedder takes cfg, change to ArcFaceEmbedder(cfg)

    # 2) Load FAISS artifacts + labels
    index, xb, labels = FaissVector.load_all(cfg.OUT_FAISS, cfg.OUT_NPY, cfg.OUT_TXT)
    fv = FaissVector(); fv.index = index
    K = int(k or getattr(cfg, "TOPK", 5))

    # 3) Read → align → (optional) preprocess
    bgr = cv2.imread(str(p), cv2.IMREAD_COLOR)
    if bgr is None:
        raise ValueError(f"Cannot read image: {p}")
    q_full = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

    q_aligned = _align(pre, q_full)
    if q_aligned is None:
        raise RuntimeError("Alignment failed (no face?)")

    # Safe pipeline with fallback (same behavior as in predict_folder)
    try:
        q_proc = _apply_pipeline(pre, q_aligned)
    except Exception as e:
        print(f"[warn] preprocessing failed, fallback to aligned: {e}")
        q_proc = q_aligned

    # 4) Embed + search
    vec = emb.embed_batch([q_proc])[0].astype(np.float32)
    hits = fv.search(vec, K)   # list[(idx, score)]
    topk_labels = [labels[i] if 0 <= i < len(labels) else f"<id:{i}>" for i, _ in hits]

    # Print Top-K
    print(f"\nTop-{K} for: {p.name}")
    for r, (i, s) in enumerate(hits, 1):
        lbl = labels[i] if 0 <= i < len(labels) else f"<id:{i}>"
        print(f"  {r:>2}. {lbl}   cos={s:.4f}")

    # 5) Viz: 6-pack if reps exist & cfg.SAVE_REPS else 3-panel
    if show:
        best_label = topk_labels[0]
        title = f"Prediction: {best_label} (cos={hits[0][1]:.4f})"

        if getattr(cfg, "SAVE_REPS", False):
            g_full, g_aligned, g_proc = _load_gallery_reps_from_disk(cfg, best_label)
        else:
            g_full = g_aligned = g_proc = None

        if g_full is not None:
            _show_sixpack(q_full, q_aligned, q_proc, g_full, g_aligned, g_proc, title)
        else:
            _show_triplet(q_full, q_aligned, q_proc, title)

    return hits, topk_labels

hits, labels = predict_one(
    r"C:\Users\syahr\Desktop\Portfolio\Face-Recognition-on-Black-Face\data\query\9202025612000001_1.png",
    k=5, show=True
)

In [ ]:
#!/usr/bin/env python3
# app/main.py
"""
Main entry
==============================

What this does
--------------
- Loads your config (SimpleConfig) and prints a quick preview.
- Creates the Preprocessor (alignment + preprocessing) and ArcFace embedder.
- Loads the FAISS index + embeddings + labels from disk.
- Runs predictions for all images inside cfg.QUERY_ROOT.

Extras (notebook & CLI)
-----------------------
- run_predict(): call from .ipynb to run the normal prediction flow.
- run_eval(): call from .ipynb to run batch evaluation (Top-1 / Top-K).
- CLI flags (--eval, --k, --no-viz) with parse_known_args() so Jupyter's --f is ignored.

Requirements (already built beforehand)
---------------------------------------
- gallery_flat.faiss
- gallery_full.npy
- gallery_ids.txt

Tips
----
- If loading the index fails, make sure you’ve built it with app/faiss_vector_build.py.
- Set cfg.SAVE_REPS=True when building to see the 6-panel visualization here.
"""

from pathlib import Path
import argparse
from typing import Any, Dict, List, Tuple

from app.config import SimpleConfig
from app.preprocess import Preprocessor
from app.arcface import ArcFaceEmbedder
from app.faiss_vec import FaissVector
from app.predict import predict_folder, BatchEvaluator


# ---------------- small utilities (non-breaking) ----------------
def _print_cfg_preview(cfg: SimpleConfig) -> None:
    print("=== Config preview ===")
    print({
        "db_dsn": cfg.DB_DSN,
        "onnx": str(cfg.ONNX_PATH),
        "out_faiss": str(getattr(cfg, "OUT_FAISS", "gallery_flat.faiss")),
        "out_npy":   str(getattr(cfg, "OUT_NPY",   "gallery_full.npy")),
        "out_txt":   str(getattr(cfg, "OUT_TXT",   "gallery_ids.txt")),
        "save_reps": getattr(cfg, "SAVE_REPS", False),
        "reps_dir":  str(getattr(cfg, "REPS_DIR",  "reps")),
        "query_dir": str(getattr(cfg, "QUERY_ROOT","data/query")),
        "topk":      getattr(cfg, "TOPK", 5),
    })
    print("======================")


def _load_faiss_artifacts(cfg: SimpleConfig) -> tuple[FaissVector, list[str], object]:
    """
    Loads FAISS index + vectors + labels using your existing loader.
    Returns (fv_wrapper, labels, xb) where fv_wrapper.index is set.
    """
    fv = FaissVector()
    try:
        index, xb, labels = FaissVector.load_all(cfg.OUT_FAISS, cfg.OUT_NPY, cfg.OUT_TXT)
    except FileNotFoundError as e:
        print(f"❌ Could not find FAISS artifacts: {e}")
        print("   → Build them first with: python -m app.faiss_vector_build")
        raise
    except Exception as e:
        print(f"❌ Failed to load FAISS artifacts: {e}")
        raise

    fv.index = index
    print(f"[ok] loaded FAISS index: {index.ntotal} vecs; labels={len(labels)}; xb={getattr(xb, 'shape', '?')}")
    return fv, labels, xb


def _normalize_eval_result(result: Any) -> Tuple[Dict[str, Any] | None, List[Dict[str, Any]]]:
    """
    Make the evaluator return shape robust:
      - summary
      - (summary, rows)
      - (summary, rows, anything_else)
    """
    if isinstance(result, tuple):
        if len(result) == 0:
            return None, []
        if len(result) == 1:
            return result[0], []
        # len >= 2
        return result[0], result[1] if isinstance(result[1], list) else []
    # single dict or None
    return (result if isinstance(result, dict) else None), []


def _print_eval_summary(summary: dict | None) -> None:
    print("\n=== Evaluation Summary ===")
    if not summary:
        print("No results to report (empty queries or evaluation failed).")
    else:
        for key, val in summary.items():
            print(f"{key}: {val}")
    print("==========================")


# ---------------- notebook-friendly helpers ----------------
def run_predict(no_viz: bool = False) -> None:
    """
    Call this from a notebook cell to run the normal prediction flow.
    Example:
        from app.main import run_predict
        run_predict()
    """
    cfg = SimpleConfig()
    if hasattr(cfg, "ensure_output_dirs"):
        cfg.ensure_output_dirs()
    _print_cfg_preview(cfg)

    pre = Preprocessor(cfg)
    emb = ArcFaceEmbedder(cfg.ONNX_PATH)  # kept as in your original code
    fv, labels, _ = _load_faiss_artifacts(cfg)

    if no_viz:
        print("[info] no_viz=True (predict_folder may still show figures if it doesn't support a flag).")
    predict_folder(cfg=cfg, pre=pre, emb=emb, fv=fv, labels=labels, outdir=Path("out"))


def run_eval(k: int | None = None, verbose: bool = True):
    """
    Call this from a notebook cell to evaluate accuracy on cfg.QUERY_ROOT.
    Returns (summary_dict, rows_list).
    Example:
        from app.main import run_eval
        summary, rows = run_eval(k=5); summary
    """
    cfg = SimpleConfig()
    if hasattr(cfg, "ensure_output_dirs"):
        cfg.ensure_output_dirs()
    _print_cfg_preview(cfg)

    pre = Preprocessor(cfg)
    emb = ArcFaceEmbedder(cfg.ONNX_PATH)  # kept consistent with your code
    fv, labels, _ = _load_faiss_artifacts(cfg)

    ev = BatchEvaluator(cfg, pre, emb, fv, labels)
    kk = k or getattr(cfg, "TOPK", 5)

    result = ev.evaluate(topk=kk, verbose=verbose)
    summary, rows = _normalize_eval_result(result)
    _print_eval_summary(summary)
    return summary, rows


# ---------------- original main flow (intact) ----------------
def main() -> None:
    # Accept notebook/CLI: ignore unknown args (e.g., Jupyter's --f=...)
    ap = argparse.ArgumentParser(description="FaceID — Predict or Evaluate")
    ap.add_argument("--eval", action="store_true", help="Run batch evaluation on cfg.QUERY_ROOT.")
    ap.add_argument("--k", type=int, default=None, help="Override K for accuracy@K (default: cfg.TOPK).")
    ap.add_argument("--no-viz", action="store_true", help="Skip visualization in prediction run.")
    args, _ = ap.parse_known_args()

    # 1) Load config and show a tiny preview
    cfg = SimpleConfig()
    if hasattr(cfg, "ensure_output_dirs"):
        cfg.ensure_output_dirs()
    _print_cfg_preview(cfg)

    # 2) Create helpers (alignment/preproc + ONNX embedder + FAISS wrapper)
    pre = Preprocessor(cfg)
    emb = ArcFaceEmbedder(cfg.ONNX_PATH)  # kept as in your original code
    fv  = FaissVector()

    # 3) Load FAISS artifacts (index, embeddings, labels)
    try:
        index, xb, labels = FaissVector.load_all(cfg.OUT_FAISS, cfg.OUT_NPY, cfg.OUT_TXT)
    except FileNotFoundError as e:
        print(f"❌ Could not find FAISS artifacts: {e}")
        print("   → Build them first with: python -m app.faiss_vector_build")
        return
    except Exception as e:
        print(f"❌ Failed to load FAISS artifacts: {e}")
        return

    fv.index = index
    print(f"[ok] loaded FAISS index: {index.ntotal} vecs; labels={len(labels)}; xb={getattr(xb, 'shape', '?')}")

    # 4) Branch: evaluation or predictions (default)
    k = args.k or getattr(cfg, "TOPK", 5)
    print(f"\n[eval] Running accuracy evaluation on '{cfg.QUERY_ROOT}' (K={k})...")
    ev = BatchEvaluator(cfg, pre, emb, fv, labels)
    result = ev.evaluate(topk=k, verbose=True)
    summary, _rows = _normalize_eval_result(result)
    _print_eval_summary(summary)

    # Default: predictions (will show 3-panel or 6-panel depending on cfg.SAVE_REPS & available reps)
    if args.no_viz:
        print("[info] --no-viz set; visual windows may still appear if predict_folder shows them.")
    predict_folder(cfg=cfg, pre=pre, emb=emb, fv=fv, labels=labels, outdir=Path("out"))


if __name__ == "__main__":
    main()
